In [2]:
import nltk
from collections import Counter
from nltk import ngrams
from Levenshtein import distance
import string
from nltk import pos_tag
from gensim.models import KeyedVectors
from collections import defaultdict

# Download necessary NLTK data
nltk.download('words')
nltk.download('brown')

from nltk.corpus import words, brown

[nltk_data] Downloading package words to /home/nathalie/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to /home/nathalie/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [3]:
# Load vocabulary and word frequencies
vocabulary = set(words.words())  # List of valid words
word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus

In [6]:
def preprocess_text(text):
    """Remove punctuation and lowercase text."""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator).lower()

In [9]:
tokens = [word.lower() for word in brown.words()]

In [11]:
model = Counter(ngrams(tokens, 3))

In [26]:
# Load vocabulary and word frequencies
# vocabulary = set(words.words())  # List of valid words
word_frequencies = Counter(brown.words())  # Word frequencies from the Brown corpus

model = Counter(ngrams(tokens, 3))

# input = 'cheking speling is complicatek'
# input = 'my mathwr is angru'
# input = 'the bes are vry fastt whn they are flyinh'
# input = 'the girl has multipe dresses'
# input = 'korrectud'
input = 'i am beginning to wamder if lonjer sentinces wuld work bether for tis prgramn'
input_preprocessed = preprocess_text(input)
words = input_preprocessed.split()

corrected_words = []
prev_word = None

for i, word in enumerate(words):
    # if word in vocabulary:
    #     corrected_words.append(word)
    #     prev_word = word
    #     continue

    word_tag = pos_tag([word])[0][1]
    print(word, word_tag)
 
    # Generate suggestions
    suggestions = [
        (vocab_word, distance(word, vocab_word), word_frequencies[vocab_word])
        for vocab_word in vocabulary
    ]
    suggestions = sorted(suggestions, key=lambda x: (x[1], -x[2]))
    suggestions = [suggestion[0] for suggestion in suggestions[0:5]]
    print(suggestions)

    # Initialize variables
    best_suggestion = suggestions[0]
    best_combined_prob = 0

    # Get the next word (if it exists)
    next_word = words[i + 1] if i + 1 < len(words) else None

    for suggestion in suggestions:
        # Calculate bigram probability with the previous word
        prob_prev = 0
        if prev_word:
            bigram_count = model.get((prev_word, suggestion), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == prev_word)
            prob_prev = bigram_count / unigram_count if unigram_count > 0 else 0

        # Calculate bigram probability with the next word
        prob_next = 0
        if next_word:
            bigram_count = model.get((suggestion, next_word), 0)
            unigram_count = sum(1 for bigram in model if bigram[0] == suggestion)
            prob_next = bigram_count / unigram_count if unigram_count > 0 else 0

        # Combine probabilities (you can adjust weights as needed)
        combined_prob = (prob_prev + prob_next) / 2

        # Update best suggestion based on combined probability
        if combined_prob > best_combined_prob:
            best_combined_prob = combined_prob
            best_suggestion = suggestion

    corrected_words.append(best_suggestion)
    prev_word = best_suggestion  # Update previous word

corrected_text = ' '.join(corrected_words)
print(corrected_text)

i NN
['i', 'a', 'in', 'is', 'it']
am VBP
['am', 'a', 'as', 'at', 'an']
beginning VBG
['beginning', 'ginning', 'unbeginning', 'rebeginning', 'binning']
to TO
['to', 'no', 'so', 'two', 'do']
wamder NN
['wander', 'warder', 'wadder', 'wader', 'water']
if IN
['if', 'of', 'in', 'is', 'it']
lonjer NN
['longer', 'lower', 'wonder', 'lover', 'locker']
sentinces NNS
['sentence', 'sentinel', 'sentencer', 'sentience', 'Bentincks']
wuld NN
['would', 'wild', 'weld', 'wold', 'suld']
work NN
['work', 'word', 'works', 'wore', 'worn']
bether NN
['better', 'bother', 'ether', 'nether', 'yether']
for IN
['for', 'or', 'For', 'far', 'form']
tis NN
['is', 'his', 'this', 'tie', 'tip']
prgramn NN
['program', 'predamn', 'grain', 'organ', 'gram']
i am beginning to wander if longer sentence would work better for is program
